In [2]:
from datetime import datetime, timedelta
import pandas as pd

from bloom.container import UseCases
from bloom.logger import logger
from thefuzz import fuzz

/Users/marthevienne/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pydantic_settings/sources.py:611: UserWarning: directory "/Users/marthevienne/.pyenv/versions/3.12.5/lib/python3.12/secrets" does not exist
  warnings.warn(f'directory "{path}" does not exist')


In [3]:
def to_coords(row: pd.Series) -> pd.Series:
    if pd.isna(row["end_position"]) is False:
        row["longitude"] = row["end_position"].x
        row["latitude"] = row["end_position"].y

    return row

In [4]:
def fuzzy_match_vessel_ais(row: any):
    match_name = fuzz.ratio(row["ship_name"], row["vessel_name"])
    match_external_marking = fuzz.ratio(row["external_marking"], row["vessel_name"])

    if match_name > 70 or match_external_marking > 70:
        return True
    return False

In [5]:
use_cases = UseCases()
db = use_cases.db()
spire_repository = use_cases.spire_ais_data_repository()
vessel_repository = use_cases.vessel_repository()

with db.session() as session:
    point_in_time = datetime(2024, 10, 25, 00, 00, 00)
    batch_limit = point_in_time + timedelta(days=3)
    # Step 1: load SPIRE batch: read from SpireAisData
    batch = spire_repository.get_all_data_between_date_bis(session, point_in_time, batch_limit)
    vessels = vessel_repository.get_vessels_list(session)

In [6]:
vessels = pd.DataFrame([o.__dict__ for o in vessels])

In [8]:
vessels_details = vessels[["mmsi", "ship_name", "external_marking", "ircs", "imo"]]

In [ ]:
ais_with_vessel_info = pd.merge(
    batch,
    vessels_details,
    how="left",
    left_on="vessel_mmsi",
    right_on="mmsi",
)

matches_ircs_imo = ais_with_vessel_info[
    (ais_with_vessel_info.vessel_callsign == ais_with_vessel_info.ircs)
    | (ais_with_vessel_info.vessel_imo == ais_with_vessel_info.imo)
]

messages_left = pd.merge(
    ais_with_vessel_info,
    matches_ircs_imo[["vessel_mmsi", "created_at"]].drop_duplicates(),
    how="left",
    indicator=True,
)

messages_left = messages_left[messages_left._merge == "left_only"]

In [18]:
messages_left[["vessel_mmsi", "vessel_imo", "vessel_callsign", "imo", "ircs"]].drop_duplicates().__len__()

53

In [105]:
batch_grouped = batch.groupby(["vessel_mmsi", "created_at"])
single_message_dates = batch_grouped.filter(lambda x: len(x) == 1)
multiple_messages_dates = batch_grouped.filter(lambda x: len(x) > 1)
multiple_messages_dates_grouped = multiple_messages_dates.groupby(
    ["vessel_mmsi", "created_at"]
)

In [ ]:
# Put aside single messages
# Work with multiple messages
## Merge with vessels and compare IRCS and IMO => put aside matches and count messages per group (mmsi, created_at)
## Work with groups of messages didn't match IMO or IRCS => match external marking and ship name with vessel name => put aside matches and count messages per group (mmsi, created_at)
## Summary messages left

In [109]:
multiple_messages_dates_with_vessel_info = pd.merge(
    multiple_messages_dates,
    vessels_details,
    how="left",
    left_on="vessel_mmsi",
    right_on="mmsi",
)

matches_ircs_imo = multiple_messages_dates_with_vessel_info[
    (
        multiple_messages_dates_with_vessel_info.vessel_callsign
        == multiple_messages_dates_with_vessel_info.ircs
    )
    & (
        multiple_messages_dates_with_vessel_info.vessel_imo
        == multiple_messages_dates_with_vessel_info.imo
    )
]

matches_ircs_imo_groups = matches_ircs_imo.groupby(["vessel_mmsi", "created_at"])
matches_ircs_imo_multiple = matches_ircs_imo_groups.filter(lambda x: len(x) > 1)
matches_ircs_imo_single = matches_ircs_imo_groups.filter(lambda x: len(x) == 1)
if matches_ircs_imo_multiple.__len__() > 0:
    logger.warning(f'The following MMSI have simultaneous multiple positions that match with IRCS and/or IMO: {matches_ircs_imo_multiple.vessel_mmsi.unique()}')

messages_to_match_with_name = pd.merge(
    multiple_messages_dates_with_vessel_info,
    matches_ircs_imo[["vessel_mmsi", "created_at"]].drop_duplicates(),
    how="left",
    indicator=True,
)

messages_to_match_with_name = messages_to_match_with_name[
    messages_to_match_with_name._merge == "left_only"
]

messages_to_match_with_name["correspondance"] = messages_to_match_with_name.apply(
    lambda x: fuzzy_match_vessel_ais(x), axis=1
)

messages_to_match_with_name_groups = messages_to_match_with_name[
    messages_to_match_with_name.correspondance == True
].groupby(["vessel_mmsi", "created_at"])
messages_to_match_with_name_multiple = messages_to_match_with_name_groups.filter(
    lambda x: len(x) > 1
)
messages_to_match_with_name_single = messages_to_match_with_name_groups.filter(
    lambda x: len(x) == 1
)
if messages_to_match_with_name_multiple.__len__() > 0:
    logger.warning(
        f"The following MMSI have simultaneous multiple positions that match with ship name and/or external marking: {messages_to_match_with_name_multiple.vessel_mmsi.unique()}"
    )

match_ais = pd.concat([matches_ircs_imo, messages_to_match_with_name_multiple])

messages_left = pd.merge(
    multiple_messages_dates_with_vessel_info,
    match_ais[["vessel_mmsi", "created_at"]].drop_duplicates(),
    how="left",
    indicator=True,
)

messages_left = messages_left[messages_left._merge == "left_only"]

if messages_left.__len__() > 0:
    logger.warning(
        f"The following MMSI have multiple messages that did not match any info: {messages_left.vessel_mmsi.unique()}"
    )

In [110]:
cleaned_batch = pd.concat(
    [single_message_dates, matches_ircs_imo_single, messages_to_match_with_name_single]
)
cleaned_batch_sorted = cleaned_batch.sort_index()
cleaned_batch_grouped = cleaned_batch_sorted.groupby("vessel_mmsi")

In [111]:
for mmsi, group in cleaned_batch_grouped:
    dt = group["position_timestamp"].diff()
    if dt[dt < timedelta(seconds=0)].__len__() > 0:
        print(mmsi)
        print(dt[dt < timedelta(seconds=0)])